<a href="https://colab.research.google.com/github/talhaanwarch/Offensive-Language-Detection/blob/master/tpu_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# a = []
# while(1):
#     a.append('1111111111111111111111111111111111111111111111')

In [0]:
#https://www.kaggle.com/sriharihumbarwadi/fine-tuning-bert-on-tpu

In [1]:
!pip install transformers --quiet
%tensorflow_version 2.x

     |████████████████████████████████| 501kB 3.5MB/s 
     |████████████████████████████████| 870kB 9.7MB/s 
     |████████████████████████████████| 3.7MB 22.0MB/s 
     |████████████████████████████████| 1.0MB 61.7MB/s 
TensorFlow 2.x selected.


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense
from transformers import TFBertModel
import pandas as pd
from transformers.tokenization_bert import BertTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0


In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.122.106.114:8470']
INFO:tensorflow:Initializing the TPU system: 10.122.106.114:8470


INFO:tensorflow:Initializing the TPU system: 10.122.106.114:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [4]:
batch_size = 8 * strategy.num_replicas_in_sync
epochs = 1

autotune = tf.data.experimental.AUTOTUNE
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

# Load data

In [11]:
train=pd.read_csv( 'https://raw.githubusercontent.com/talhaanwarch/Offensive-Language-Detection/master/Data/olid-training-v1.0.tsv',sep="\t")
train=train[['tweet','subtask_a']]
train.head()

,tweet,subtask_a
0,@USER She should ask a few native Americans wh...,OFF
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,Amazon is investigating Chinese employees who ...,NOT
3,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,@USER @USER Obama wanted liberals &amp; illega...,NOT


In [0]:
mappping={'OFF':1,'NOT':0}
train['subtask_a']=train['subtask_a'].map(mappping)

In [13]:

train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('URL', '')
train['tweet']= train['tweet'].str.lower()

train.head()

,tweet,subtask_a
0,she should ask a few native americans what th...,1
1,go home you’re drunk!!! #maga #trump 👊🇺🇸👊,1
2,amazon is investigating chinese employees who ...,0
3,"someone should'vetaken"" this piece of shit to...",1
4,obama wanted liberals &amp; illegals to move...,0


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train['tweet'],train['subtask_a'],test_size=0.3)

In [16]:
X_train.head()

8350                             devil worshippers unite!
7433              he is making faces like peter strokz...
2220                                 evil in a white robe
6466        by any chance did you sexually assault gir...
8854                                     and he was right
Name: tweet, dtype: object

In [17]:
with strategy.scope():
    tweets = tf.keras.Input(shape=(128,), dtype=tf.int32)
    bert = TFBertModel.from_pretrained(pretrained_weights)
    tweets_hidden_mean = tf.reduce_mean(bert(tweets)[0], axis=1)
    x = Dense(units=64,activation='relu')(tweets_hidden_mean)
    logits = Dense(units=1, name='logits', activation='sigmoid')(x)
    model = tf.keras.Model(inputs=[tweets], outputs=[logits])

In [0]:
encoded_train = [tokenizer.encode(text, max_length=128, pad_to_max_length=True) for text in X_train]
encoded_test = [tokenizer.encode(text, max_length=128, pad_to_max_length=True) for text in X_test]

In [19]:
len(encoded_train)

9268

In [0]:
def make_tfdataset(x, y):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.shuffle(128)
    dataset = dataset.repeat(epochs)
    dataset = dataset.prefetch(autotune)
    return dataset

In [0]:
with strategy.scope():
    train_dataset = make_tfdataset(encoded_train, np.array(y_train))
    test_dataset = make_tfdataset(encoded_test, y_test)

In [0]:
with strategy.scope():
    metrics_list = [tf.metrics.BinaryAccuracy(), tf.metrics.Precision(), tf.metrics.Recall()]
    model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=False),
                  metrics=metrics_list,
                  optimizer=tf.optimizers.Adam(1e-4))
model.fit(train_dataset, epochs=1, validation_data=test_dataset)  # change steps_per_epoch when training

Train for 144 steps, validate for 62 steps
